<a href="https://colab.research.google.com/github/hiris25/TEVC-analysis-scripts/blob/master/DR_antagonists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TEVC - Dose Response - Antagonists**

## Set up libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.optimize as opt
from scipy import stats
import glob as glob
import matplotlib.ticker as ticker

%config InlineBackend.figure_format = 'retina'

## Import and prepare data

### Import Files

Import all files from specified folder and combine into one dataframe called 'df'

In [ ]:
path = r'folderlocation' # use your path
all_files = glob.glob(path + "/*extension.dat") # use your extension

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep='\t') # for .dat files you must specify tab separation sep='\t'
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

(84, 64)

### Remove excess coloumns and clean data
Here we are keeping only the relevant information, you can add or remove coloumns as you like.
We will also be standardising the units and generally cleaning up the data

In [ ]:
data = df[['Inj. ID 1'] + ['Well'] + ['Buffer'] + ['Comp. 1'] + ['conc. 1'] + ['unit 1'] + ['Minimum'] + ['Baseline Average'] + ['Start Date'] + ['Comp. 2'] + ['conc. 2'] + ['unit 2']]

data['date'] = data['Start Date'].str.extract('(../../....)', expand=True)

Transform units all to uM scale

In [ ]:
conditions = [
    (data['unit 1'] == 'mM'),
    (data['unit 1'] == 'µM'),
    (data['unit 1'] == 'nM')]
choices = ['1000', '1', '0.001']
data['factor'] = np.select(conditions, choices, default='0')

data['factor']=data.factor.astype(float)

data['Comp1_Concentration_µM'] = data['conc. 1'] * data['factor']

conditions = [
    (data['unit 2'] == 'mM'),
    (data['unit 2'] == 'µM'),
    (data['unit 2'] == 'nM')]
choices = ['1000', '1', '0.001']
data['factor'] = np.select(conditions, choices, default='0')

data['factor']=data.factor.astype(float)

data['Comp2_Concentration_µM'] = data['conc. 2'] * data['factor']

data =  data[['Inj. ID 1'] + ['Well'] + ['Buffer'] + ['Comp. 1'] + ['Comp. 2'] + ['Minimum'] + ['Comp2_Concentration_µM'] + ['date'] + ['Comp1_Concentration_µM']]
data.reset_index(drop=True)    

In [ ]:
#decide if comp1 or comp2 is agonist or antagonsit

data['log_Agonist_µM'] = np.log(data['Comp2_Concentration_µM'])
data['log_Antagonist_µM'] = np.log(data['Comp1_Concentration_µM'])

data['Antagonist'] = data['Comp. 1']

data['Agonist'] = data['Comp. 2']

data['Response'] = data['Minimum']
data['Agonist_µM'] = data['Comp2_Concentration_µM']
data['Antagonist_µM'] = data['Comp1_Concentration_µM']

data = data.drop(columns=['Comp. 2'])
data = data.drop(columns=['Comp. 1'])
data = data.drop(columns=['Comp2_Concentration_µM'])
data = data.drop(columns=['Comp1_Concentration_µM'])

## Transform data to I/Imax for each oocyte and each compound(optional)

comment out if not you do **NOT** want to normalise data

In [ ]:
minvalues = data.groupby(['Well', 'date', 'Antagonist'])["Minimum"].min()

norm = data.Minimum / data.groupby(['Well', 'Antagonist']).Minimum.transform(np.min)

data['Response'] = norm

data.drop(columns=['Minimum', 'Buffer'])

In [ ]:
# Check data (skip if you did I/Imax)
#data = data.drop(columns=['Minimum'])

## Exclude outliers and whole oocytes

### Exclude specific oocytes (optional)
Use the code below to exclude data for indivdiual oocytes (rows)

In [ ]:
#data = data[~data['Well'].isin(['A1'])]

(84, 12)

## Calculate means and export data

### Export as .csv
comment out if you do not want to export .csv
change path and file name before running

exports grouped mean without outliers, std, N into a new .csv file

Includes Agonist and log agonist concentrations in uM

In [ ]:
export = data.groupby(['Antagonist', 'Antagonist_µM'])

export = export['Response'].describe()

export.head(15)

export.to_csv(r'filelocation/filename.csv')